<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/01_P6_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [1]:
!pip install feature_engine


     |████████████████████████████████| 9.5MB 7.1MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
!pip install plotly

In [3]:
!pip install chart-studio

     |████████████████████████████████| 71kB 3.2MB/s 


In [4]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec #Word Embedding

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


SELECT Id, Body, Title, Tags

FROM Posts

WHERE Score >= 3 AND PostTypeId = 1

ORDER BY RAND()
 
ASC OFFSET 0 ROWS FETCH NEXT 50000 ROWS ONLY; de 0 à 50000

## **2.Data**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df_analysis_01 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_01.csv')
#df_analysis_02 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_02.csv')
#df_analysis_03 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_03.csv')
#frames = [df_analysis_01, df_analysis_02, df_analysis_03]
#df_analysis = pd.concat(frames)
#df_sample = df_analysis.sample(n=40000)
#df_sample.to_csv('df_sample.csv')

In [7]:
df_sample = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/df_sample.csv')
df_sample = df_sample.drop(['Unnamed: 0'], axis=1)
df_sample.head()

,Id,Body,Title,Tags,CreationDate
0,48060830,<p>I am trying to create a very simple system ...,Create System Tray Icon from Webdings font,<c#><.net><windows><icons>,2018-01-02 12:15:37
1,57451083,"<p><a href=""http://eel.is/c++draft/basic.exec#...",Intent of [basic.execution] p5 sentence 2,<c++><language-lawyer><full-expression>,2019-08-11 14:59:51
2,56279423,<p>I use the MongoDB aggregation API to aggreg...,MongoDB: Reduce array of objects into a single...,<mongodb><mongodb-query><aggregation-framework>,2019-05-23 16:24:08
3,48326694,<p>I wonder if its possible to validate a arra...,Laravel validate array of objects,<laravel>,2018-01-18 17:03:31
4,58501086,<p>I am trying to load the save weights using ...,ValueError: Unknown activation function:swish_...,<python><tensorflow><keras><activation-function>,2019-10-22 09:25:59


In [8]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            40000 non-null  int64 
 1   Body          40000 non-null  object
 2   Title         40000 non-null  object
 3   Tags          40000 non-null  object
 4   CreationDate  40000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [9]:
df_sample = df_sample.loc[:, ['Id', 'Title', 'Body', 'Tags']]
df_sample = df_sample.set_index('Id').reset_index(drop=True)
df_sample.head()

,Title,Body,Tags
0,Create System Tray Icon from Webdings font,<p>I am trying to create a very simple system ...,<c#><.net><windows><icons>
1,Intent of [basic.execution] p5 sentence 2,"<p><a href=""http://eel.is/c++draft/basic.exec#...",<c++><language-lawyer><full-expression>
2,MongoDB: Reduce array of objects into a single...,<p>I use the MongoDB aggregation API to aggreg...,<mongodb><mongodb-query><aggregation-framework>
3,Laravel validate array of objects,<p>I wonder if its possible to validate a arra...,<laravel>
4,ValueError: Unknown activation function:swish_...,<p>I am trying to load the save weights using ...,<python><tensorflow><keras><activation-function>


## **3.Preprocessing**

In [10]:
df_sample.duplicated(subset=["Body", "Title"]).sum()

0

In [11]:
example_body = df_sample.Body.iloc[0]

In [12]:
example_body

'<p>I am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border.  To do that, we are using the Webdings font.  "n" in Webdings is just a plain circle.</p>\n\n<p>What I\'m currently doing is almost there, but on some PC\'s (yet not all) it ends up having a choppy, ugly black border around it:</p>\n\n<p><a href="https://i.stack.imgur.com/2rwwx.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/2rwwx.png" alt="enter image description here"></a></p>\n\n<p>Here\'s what I\'ve got:</p>\n\n<pre><code>protected static Icon GetTrayIconFromCache(Color statusColor)\n{\n    Bitmap bmp = new Bitmap(16,16);                            \n    Graphics circleGraphic = Graphics.FromImage(bmp);\n    circleGraphic.DrawString("n", new Font("Webdings", 12F, FontStyle.Regular), Brushes.White, -3f, -2f);\n    circleGraphic.DrawString("n", new Font("Webdings", 9F, FontStyle.Regular), new SolidBrush(statusColor), 0f, -1f);\n    Icon ico

In [13]:
example_title = df_sample.Title.iloc[0]

In [14]:
example_title

'Create System Tray Icon from Webdings font'

In [15]:
#Merge Title and Body
df_sample['Title_Body'] = df_sample['Title'] + ' ' + df_sample['Body']

In [16]:
df_sample.head()


,Title,Body,Tags,Title_Body
0,Create System Tray Icon from Webdings font,<p>I am trying to create a very simple system ...,<c#><.net><windows><icons>,Create System Tray Icon from Webdings font <p>...
1,Intent of [basic.execution] p5 sentence 2,"<p><a href=""http://eel.is/c++draft/basic.exec#...",<c++><language-lawyer><full-expression>,Intent of [basic.execution] p5 sentence 2 <p><...
2,MongoDB: Reduce array of objects into a single...,<p>I use the MongoDB aggregation API to aggreg...,<mongodb><mongodb-query><aggregation-framework>,MongoDB: Reduce array of objects into a single...
3,Laravel validate array of objects,<p>I wonder if its possible to validate a arra...,<laravel>,Laravel validate array of objects <p>I wonder ...
4,ValueError: Unknown activation function:swish_...,<p>I am trying to load the save weights using ...,<python><tensorflow><keras><activation-function>,ValueError: Unknown activation function:swish_...


In [17]:
variables_reorganization = ['Title_Body', 'Tags']
df_sample = df_sample[['Title_Body', 'Tags']]

In [18]:
df_sample.head()

,Title_Body,Tags
0,Create System Tray Icon from Webdings font <p>...,<c#><.net><windows><icons>
1,Intent of [basic.execution] p5 sentence 2 <p><...,<c++><language-lawyer><full-expression>
2,MongoDB: Reduce array of objects into a single...,<mongodb><mongodb-query><aggregation-framework>
3,Laravel validate array of objects <p>I wonder ...,<laravel>
4,ValueError: Unknown activation function:swish_...,<python><tensorflow><keras><activation-function>


In [19]:
df_sample.Title_Body.iloc[0]

'Create System Tray Icon from Webdings font <p>I am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border.  To do that, we are using the Webdings font.  "n" in Webdings is just a plain circle.</p>\n\n<p>What I\'m currently doing is almost there, but on some PC\'s (yet not all) it ends up having a choppy, ugly black border around it:</p>\n\n<p><a href="https://i.stack.imgur.com/2rwwx.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/2rwwx.png" alt="enter image description here"></a></p>\n\n<p>Here\'s what I\'ve got:</p>\n\n<pre><code>protected static Icon GetTrayIconFromCache(Color statusColor)\n{\n    Bitmap bmp = new Bitmap(16,16);                            \n    Graphics circleGraphic = Graphics.FromImage(bmp);\n    circleGraphic.DrawString("n", new Font("Webdings", 12F, FontStyle.Regular), Brushes.White, -3f, -2f);\n    circleGraphic.DrawString("n", new Font("Webdings", 9F, FontStyle.Regular), new Solid

### **HTML**

In [20]:
#HTML characters removing
def remove_html(body):
  html_regex = re.compile('<.*?>') #Compile regular expresions
  return re.sub(html_regex, ' ', str(body)) # Replace regex by ' '

In [21]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_html)
df_sample.head()

,Title_Body,Tags
0,Create System Tray Icon from Webdings font I ...,<c#><.net><windows><icons>
1,Intent of [basic.execution] p5 sentence 2 [b...,<c++><language-lawyer><full-expression>
2,MongoDB: Reduce array of objects into a single...,<mongodb><mongodb-query><aggregation-framework>
3,Laravel validate array of objects I wonder if...,<laravel>
4,ValueError: Unknown activation function:swish_...,<python><tensorflow><keras><activation-function>


In [22]:
df_sample.Title_Body.iloc[0]

'Create System Tray Icon from Webdings font  I am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border.  To do that, we are using the Webdings font.  "n" in Webdings is just a plain circle. \n\n What I\'m currently doing is almost there, but on some PC\'s (yet not all) it ends up having a choppy, ugly black border around it: \n\n     \n\n Here\'s what I\'ve got: \n\n  protected static Icon GetTrayIconFromCache(Color statusColor)\n{\n    Bitmap bmp = new Bitmap(16,16);                            \n    Graphics circleGraphic = Graphics.FromImage(bmp);\n    circleGraphic.DrawString("n", new Font("Webdings", 12F, FontStyle.Regular), Brushes.White, -3f, -2f);\n    circleGraphic.DrawString("n", new Font("Webdings", 9F, FontStyle.Regular), new SolidBrush(statusColor), 0f, -1f);\n    Icon ico = Icon.FromHandle((bmp).GetHicon());\n    return ico;\n}\n  \n\n No matter what I try, I can\'t get rid of those ugly black dots around the 

### **URL**

In [23]:
#URL removing
def remove_url(body):
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
  return re.sub(url_regex, ' ', str(body))  

In [24]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_url)
df_sample.head()

,Title_Body,Tags
0,Create System Tray Icon from Webdings font I ...,<c#><.net><windows><icons>
1,Intent of [basic.execution] p5 sentence 2 [b...,<c++><language-lawyer><full-expression>
2,MongoDB: Reduce array of objects into a single...,<mongodb><mongodb-query><aggregation-framework>
3,Laravel validate array of objects I wonder if...,<laravel>
4,ValueError: Unknown activation function:swish_...,<python><tensorflow><keras><activation-function>


In [25]:
df_sample.Title_Body.iloc[0]

'Create System Tray Icon from Webdings font  I am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border.  To do that, we are using the Webdings font.  "n" in Webdings is just a plain circle. \n\n What I\'m currently doing is almost there, but on some PC\'s (yet not all) it ends up having a choppy, ugly black border around it: \n\n     \n\n Here\'s what I\'ve got: \n\n  protected static Icon GetTrayIconFromCache(Color statusColor)\n{\n    Bitmap bmp = new Bitmap(16,16);                            \n    Graphics circleGraphic = Graphics.FromImage(bmp);\n    circleGraphic.DrawString("n", new Font("Webdings", 12F, FontStyle.Regular), Brushes.White, -3f, -2f);\n    circleGraphic.DrawString("n", new Font("Webdings", 9F, FontStyle.Regular), new SolidBrush(statusColor), 0f, -1f);\n    Icon ico = Icon.FromHandle((bmp).GetHicon());\n    return ico;\n}\n  \n\n No matter what I try, I can\'t get rid of those ugly black dots around the 

### **Punctuation**

In [26]:
#Punctuation removing
def remove_punc(body):
  clean_text = re.sub(r'[?|!|"|:|=|_|{|}|[|]|-|$|%|^|&|]',r' ',str(body))
  clean_text = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→]',r' ', clean_text)
  return clean_text

In [27]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_punc)

In [28]:
df_sample.Title_Body.iloc[0]

" Create System Tray Icon from Webdings font  I am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border   To do that  we are using the Webdings font    n  in Webdings is just a plain circle  \n\n What I'm currently doing is almost there  but on some PC's  yet not all  it ends up having a choppy  ugly black border around it  \n\n     \n\n Here's what I've got  \n\n  protected static Icon GetTrayIconFromCache Color statusColor \n \n    Bitmap bmp   new Bitmap 16 16                              \n    Graphics circleGraphic   Graphics FromImage bmp  \n    circleGraphic DrawString  n   new Font  Webdings   12F  FontStyle Regular   Brushes White   3f   2f  \n    circleGraphic DrawString  n   new Font  Webdings   9F  FontStyle Regular   new SolidBrush statusColor   0f   1f  \n    Icon ico   Icon FromHandle  bmp  GetHicon    \n    return ico \n \n  \n\n No matter what I try  I can't get rid of those ugly black dots around the outs

### **Other characters**

In [29]:
#All characters removing except letters, numbers 
#Lower
def remove_other(body):
  text = str(body)
  clean_text = re.sub(r"[^a-zA-Z0-9#+-]", " ", text.lower())
  return clean_text

In [30]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_other)
df_sample.head()

,Title_Body,Tags
0,create system tray icon from webdings font i...,<c#><.net><windows><icons>
1,intent of basic execution p5 sentence 2 ...,<c++><language-lawyer><full-expression>
2,mongodb reduce array of objects into a singl...,<mongodb><mongodb-query><aggregation-framework>
3,laravel validate array of objects i wonder i...,<laravel>
4,valueerror unknown activation function swish...,<python><tensorflow><keras><activation-function>


In [31]:
df_sample.Title_Body.iloc[0]

' create system tray icon from webdings font  i am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border   to do that  we are using the webdings font    n  in webdings is just a plain circle     what i m currently doing is almost there  but on some pc s  yet not all  it ends up having a choppy  ugly black border around it            here s what i ve got      protected static icon gettrayiconfromcache color statuscolor        bitmap bmp   new bitmap 16 16                                   graphics circlegraphic   graphics fromimage bmp       circlegraphic drawstring  n   new font  webdings   12f  fontstyle regular   brushes white   3f   2f       circlegraphic drawstring  n   new font  webdings   9f  fontstyle regular   new solidbrush statuscolor   0f   1f       icon ico   icon fromhandle  bmp  gethicon         return ico         no matter what i try  i can t get rid of those ugly black dots around the outside of the circle  

### **Space**

In [32]:
#Space removing
def remove_space(body):
  return ' '.join(str(body).split()) 

In [33]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_space)

In [34]:
df_sample.Title_Body.iloc[0]

'create system tray icon from webdings font i am trying to create a very simple system tray icon that is nothing but a dynamically colored circle with a white border to do that we are using the webdings font n in webdings is just a plain circle what i m currently doing is almost there but on some pc s yet not all it ends up having a choppy ugly black border around it here s what i ve got protected static icon gettrayiconfromcache color statuscolor bitmap bmp new bitmap 16 16 graphics circlegraphic graphics fromimage bmp circlegraphic drawstring n new font webdings 12f fontstyle regular brushes white 3f 2f circlegraphic drawstring n new font webdings 9f fontstyle regular new solidbrush statuscolor 0f 1f icon ico icon fromhandle bmp gethicon return ico no matter what i try i can t get rid of those ugly black dots around the outside of the circle they don t show up for everyone some of the developers don t see it and it looks crisp and clean we ve not yet figured out what the commonality 

In [35]:
#df_analysis['Title_Body_count'] = df_analysis['Title_Body'].apply(lambda text: len(text.split(" ")))

In [36]:
df_sample.head()

,Title_Body,Tags
0,create system tray icon from webdings font i a...,<c#><.net><windows><icons>
1,intent of basic execution p5 sentence 2 basic ...,<c++><language-lawyer><full-expression>
2,mongodb reduce array of objects into a single ...,<mongodb><mongodb-query><aggregation-framework>
3,laravel validate array of objects i wonder if ...,<laravel>
4,valueerror unknown activation function swish a...,<python><tensorflow><keras><activation-function>


### **StopWords**

In [37]:
#Tokenization
#nltk.download("punkt")
#from nltk.tokenize import word_tokenize

In [38]:
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import EnglishStemmer
lemma = WordNetLemmatizer().lemmatize
ps = PorterStemmer()
stemmer = EnglishStemmer()

In [39]:
set_alpha = set(['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y' 'z'])

In [40]:
def remove_stopwords(text):
    text = str(text)
    text = " ".join(word for word in text.split() if word not in gensim.parsing.preprocessing.STOPWORDS and word not in stop_words and word not in set_alpha )
    return text

In [41]:
df_sample['Title_Body'] = df_sample['Title_Body'].apply(remove_stopwords)

In [42]:
df_sample.Title_Body.iloc[0]

'create tray icon webdings font trying create simple tray icon dynamically colored circle white border webdings font webdings plain circle currently pc ends choppy ugly black border got protected static icon gettrayiconfromcache color statuscolor bitmap bmp new bitmap 16 16 graphics circlegraphic graphics fromimage bmp circlegraphic drawstring new font webdings 12f fontstyle regular brushes white 3f 2f circlegraphic drawstring new font webdings 9f fontstyle regular new solidbrush statuscolor 0f 1f icon ico icon fromhandle bmp gethicon return ico matter try rid ugly black dots outside circle developers looks crisp clean figured commonality pc looks good better way'

In [43]:
df_sample.head()

,Title_Body,Tags
0,create tray icon webdings font trying create s...,<c#><.net><windows><icons>
1,intent basic execution p5 sentence 2 basic exe...,<c++><language-lawyer><full-expression>
2,mongodb reduce array objects single object com...,<mongodb><mongodb-query><aggregation-framework>
3,laravel validate array objects wonder possible...,<laravel>
4,valueerror unknown activation function swish a...,<python><tensorflow><keras><activation-function>


## **Tokenization**

### **Question**

In [44]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [45]:
def tokenize(document):
  tokens = [lemma(w) for w in document.split() if w.isalpha()]
  return tokens

In [46]:
df_sample['Title_Body_Tokens'] = df_sample['Title_Body'].apply(tokenize)

In [47]:
df_sample.head()

,Title_Body,Tags,Title_Body_Tokens
0,create tray icon webdings font trying create s...,<c#><.net><windows><icons>,"[create, tray, icon, webdings, font, trying, c..."
1,intent basic execution p5 sentence 2 basic exe...,<c++><language-lawyer><full-expression>,"[intent, basic, execution, sentence, basic, ex..."
2,mongodb reduce array objects single object com...,<mongodb><mongodb-query><aggregation-framework>,"[mongodb, reduce, array, object, single, objec..."
3,laravel validate array objects wonder possible...,<laravel>,"[laravel, validate, array, object, wonder, pos..."
4,valueerror unknown activation function swish a...,<python><tensorflow><keras><activation-function>,"[valueerror, unknown, activation, function, sw..."


## **Tags**

In [48]:
from ast import literal_eval

In [49]:
df_sample.Tags.iloc[0]

'<c#><.net><windows><icons>'

In [50]:
# convert data to list
to_list = {'><': "','",'<':"['", '>':"']"}
for key, value in to_list.items():
  df_sample['Tags'] = df_sample['Tags'].str.replace(key, value)

In [51]:
df_sample_tok = df_sample
df_sample_tok.head()

,Title_Body,Tags,Title_Body_Tokens
0,create tray icon webdings font trying create s...,"['c#','.net','windows','icons']","[create, tray, icon, webdings, font, trying, c..."
1,intent basic execution p5 sentence 2 basic exe...,"['c++','language-lawyer','full-expression']","[intent, basic, execution, sentence, basic, ex..."
2,mongodb reduce array objects single object com...,"['mongodb','mongodb-query','aggregation-framew...","[mongodb, reduce, array, object, single, objec..."
3,laravel validate array objects wonder possible...,['laravel'],"[laravel, validate, array, object, wonder, pos..."
4,valueerror unknown activation function swish a...,"['python','tensorflow','keras','activation-fun...","[valueerror, unknown, activation, function, sw..."


In [52]:
def to_list(df,column):
  df[column] = df[column].apply(literal_eval)
  return df

In [53]:
df_sample_list = to_list(df_sample_tok, 'Tags')

In [54]:
reorganization = ['Title_Body', 'Title_Body_Tokens', 'Tags']
df_sample_list = df_sample_list[reorganization]
df_sample_list.head()


,Title_Body,Title_Body_Tokens,Tags
0,create tray icon webdings font trying create s...,"[create, tray, icon, webdings, font, trying, c...","[c#, .net, windows, icons]"
1,intent basic execution p5 sentence 2 basic exe...,"[intent, basic, execution, sentence, basic, ex...","[c++, language-lawyer, full-expression]"
2,mongodb reduce array objects single object com...,"[mongodb, reduce, array, object, single, objec...","[mongodb, mongodb-query, aggregation-framework]"
3,laravel validate array objects wonder possible...,"[laravel, validate, array, object, wonder, pos...",[laravel]
4,valueerror unknown activation function swish a...,"[valueerror, unknown, activation, function, sw...","[python, tensorflow, keras, activation-function]"


In [55]:
#df_sample_list.to_csv('df_sample_list.csv')

In [56]:
#df_mod = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/df_sample_list.csv')
#df_mod = df_mod.iloc[:, 1:]
#df_mod.head()

,Title_Body,Title_Body_Tokens,Tags
0,create tray icon webdings font trying create s...,"['create', 'tray', 'icon', 'webdings', 'font',...","['c#', '.net', 'windows', 'icons']"
1,intent basic execution p5 sentence 2 basic exe...,"['intent', 'basic', 'execution', 'sentence', '...","['c++', 'language-lawyer', 'full-expression']"
2,mongodb reduce array objects single object com...,"['mongodb', 'reduce', 'array', 'object', 'sing...","['mongodb', 'mongodb-query', 'aggregation-fram..."
3,laravel validate array objects wonder possible...,"['laravel', 'validate', 'array', 'object', 'wo...",['laravel']
4,valueerror unknown activation function swish a...,"['valueerror', 'unknown', 'activation', 'funct...","['python', 'tensorflow', 'keras', 'activation-..."


In [99]:
#questions2 = questions1.apply(tokenize)
#questions2